In [1]:
import os, re, sys

import pandas as pd
import numpy as np
# from tkinter.filedialog import askdirectory

In [2]:
def alphanumeric_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

def format_results(dataframe, files, timepoints, results_dir, csv_name):
    dataframe.columns = files
    dataframe['Timepoint'] = range(1,(timepoints + 1))
    dataframe = dataframe.set_index('Timepoint')
    dataframe.to_csv(os.path.join(results_dir, csv_name))

def analyze(df,timepoints,pct_list,mk_list,pplt_list):
    n = 1
    while n <= timepoints: 
        df2 = df.loc[n].reset_index()
        df2 = df2.loc[df2['Area'] > 100]
        
        # Circ. for FLMK
        mk = df2.loc[df2["Round"] > 0.4, 'Area']
        mk_area = mk.sum()
        
        # Circ. for FLMK
        pplt = df2.loc[df2["Round"] < 0.4, 'Area']
        pplt_area = pplt.sum()
        
        try:
            pct = len(pplt)/len(mk) * 100
        except ZeroDivisionError:
            pct = 0.00
        
        pct_list.append(pct)      
        mk_list.append(mk_area)
        pplt_list.append(pplt_area)

        n += 1      
        if n > timepoints:
            df3 = pd.DataFrame(
                {'Pct': pct_list,
                 'MK_Area': mk_list,
                 'Pplt_Area': pplt_list})
        
    return df3

In [3]:
DIR = 'C:\\Users\\Tech\\Desktop\\flMK_IJ'
# os.chdir(DIR)
files = alphanumeric_sort(os.listdir(DIR))

In [4]:
timepoints = 25

In [5]:
final_df = pd.DataFrame()

for file in files:
    
    df = pd.read_csv(DIR + os.sep + file, usecols = ["Area", "Round", "Slice"], index_col = ["Slice"])
    
    pct_list = []
    mk_list = []
    pplt_list = []
    
    frame = analyze(df,timepoints,pct_list,mk_list,pplt_list) 
    final_df = pd.concat([final_df,frame],axis=1)

format_output_csv(final_df.iloc[:,::3], files, timepoints, DIR, "Pct_Pplt_Prod.csv")
format_output_csv(final_df.iloc[:,1::3], files, timepoints, DIR, "Area_MK.csv")
format_output_csv(final_df.iloc[:,2::3], files, timepoints, DIR, "Area_Pplt.csv")

print("Done!")

Done!
